# Getting Started

This first workbook will introduce several new concepts:
 * Cloning code from github
 * Jupyter notebooks
 * Working with syzygy
 * Accessing the Canvas API
 

## Getting and running this code
run this command to clone the git repository:

    git clone <repo>

There's a setup notebook you will need to run before this one (it's called setup).

This notebook expects to find a file called `.canvas_api_key` that contains your API key (and nothing else). You will need to create that (in the same directory as this file) from the command line in jupyter.

First, create a Canvas API key through Canvas.

Second, save the API key to a file. from the jupyter hub page choose "New" and then "Terminal" to get an interactive terminal session. Use this command: 

    echo "replace this with your Canvas API key" >> .canvas_api_key

## Working with Jupyter notebooks
There's a great tutorial made available along with syzygy: https://intro.syzygy.ca

If you've never used Jupyter notebooks before, that's a great place to get started.

There are many shortcuts to make interacting with a notebook easier, for example you can run just the current cell (rather than re-running the entire notebook) with `shift + enter`

## Working with the Canvas API

This notebook is used to export data from Canvas. The data we export will be saved in `json` format.

It contains examples using the `canvasapi` python package (thanks to UCF!). Most (but not all) API endpoints from Canvas are available in the `canvasapi` package.

Useful links about the Canvas API:
 * https://github.com/ucfopen/canvasapi
 * https://canvas.instructure.com/doc/api/

In [ ]:
from canvasapi import Canvas
import json

In [ ]:
#setup some global variables for the script

# load API_KEY from a file
# we doing this so that we can share this code without sharing our secret keys
API_KEY=open(".canvas_api_key").readline().rstrip('\n')
API_URL = "https://canvas.ubc.ca"

COURSE_NUM = 30739 # HackUBC course ID

In [ ]:
# Here we create the canvas python object, we use that to interact with the Canvas API
canvas = Canvas(API_URL, API_KEY)

# this should print a json representation of your Canvas user information
print(json.dumps(json.loads(canvas.get_user('self').to_json()),sort_keys=True, indent=4))

## A note about Canvas discussions
the API divides discussion content/posts into three different types:

* `discussion topics`: The individual discussion forums
* `discussion entries`: The top level entries in the discussion. Each is an original response to the discussion topic.
* `discussion replies`: The response to each entry. In Canvas, these are displayed indented and nested under the corresponding discussion entry.

In [ ]:
# get all the discussion topics

# many methods of the Canvas API load data lazily (when needed)
# to force retreival of all items, we can wrap the request in a list()
all_topics = list(canvas.get_course(COURSE_NUM).get_discussion_topics())

In [ ]:
# print all the discussion topic names and their total number of posts
for t in all_topics:
    print(t.title, t.discussion_subentry_count)
    #note, this is the *full* number of posts in the topic


In [ ]:
# print all the posts in the largest discussion

largest = sorted(all_topics, key=lambda x: x.discussion_subentry_count, reverse=True)[0]

print (largest.title, len(list(largest.get_topic_entries())))
#note: this is only the *top-level* entries in the discussion, 
# not all replies to those entries

In [ ]:
# get all the top-level entries
# notice that this is different from the number of posts above
all_entries = []
for t in all_topics:
    topic_entries = list(t.get_topic_entries())
    all_entries.extend(topic_entries)
print("total number of top-level entries: ", len(all_entries))

In [ ]:
# get all the replies
# again, notice that this is different from the number of posts above
all_replies = []
for e in all_entries:
    entry_replies = list(e.get_replies())
    all_replies.extend(entry_replies)
print("total number of replies: ",len(all_replies))

In [ ]:
# get the number of replies for a each discussion entry
for topic in largest.get_topic_entries():
  print(topic, len(list(topic.get_replies())))

In [ ]:
# this is an example of how we can 'pretty print' the json representation of an object
# in this case, we are printing the details of the largest discussion topic
print(json.dumps(json.loads(largest.to_json()),indent=4))

In [ ]:
# here we are saving all the discussion data to .json files
# you can open these files from the jupyter hub to examine their contents
# or you can interactively explore the python objects from within this notebook
with open('all_topics.json', 'w') as f:
    json.dump(list(map(lambda x: json.loads(x.to_json()), all_topics)), f)
    #json.dump(all_topics, f)

with open('all_entries.json', 'w') as f:
    json.dump(list(map(lambda x: json.loads(x.to_json()), all_entries)), f)

with open('all_replies.json', 'w') as f:
    json.dump(list(map(lambda x: json.loads(x.to_json()), all_replies)), f)


## What next?

Create some additional cells below here (or above!) and try to interact with some of the results from the Canvas API.

Some things to try:
 * Extract just your username from the Canvas `get_user` api
 * Print all the details of a specific discussion topic, entry, or reply
 * Find all the topics/entires/replies that you made
 * Find every reply to a specific user (such as yourself)
 * Look at the documentation and try using a different API endpoint